# Use case introduction 📰

In this notebook, we'll explore the use case of extracting structured information from news articles.

In short, given a news article, the goal is to extract the following information:
- The title of the article
- A summary of the article
- Whether the article is about business
- For each mentioned business
  - The name of the business
  - The likely stock price change (increase, decrease, none)
  - Reason for the stock price change
  - Relevant substring supporting the reason


## Exploring the data

We are using a dataset of BBC news articles for a specific month (see `get_bbc_news_sample` function below). The dataset contains the following columns: 
- `article`: The article as we would see it on the website.
- `title`: The title of the article.
- `is_business`: Whether the article is about business.
- `description`: A short summary of the article.

As you see, we start with a *labeled* data set, as is important for many use cases. If this is not available in your case, it may be worth to label a few samples to get started.

In [1]:
from llmops_training.news_reader.data import get_bbc_news_sample

In [2]:
data = get_bbc_news_sample()
data

'_ssl.c:993: The handshake operation timed out' thrown while requesting HEAD https://huggingface.co/datasets/RealTimeData/bbc_news_alltime/resolve/main/README.md
Retrying in 1s [Retry 1/5].
'_ssl.c:993: The handshake operation timed out' thrown while requesting HEAD https://huggingface.co/datasets/RealTimeData/bbc_news_alltime/resolve/main/README.md
Retrying in 2s [Retry 2/5].
'_ssl.c:993: The handshake operation timed out' thrown while requesting HEAD https://huggingface.co/datasets/RealTimeData/bbc_news_alltime/resolve/main/README.md
Retrying in 4s [Retry 3/5].


KeyboardInterrupt: 

Let's print the first article which is about business:

In [ ]:
article = data[data["is_business"]].iloc[0].article
print(article)

In [4]:
article = """The US firm behind the Roomba smart vacuum cleaner, iRobot, has filed for bankruptcy protection after facing competition from Chinese rivals and being hit by tariffs.
Under the so-called pre-packaged Chapter 11 process, the main manufacturer of its devices, Shenzhen-based Picea Robotics, will take ownership of the firm.

The tough commercial landscape had forced iRobot to cut its prices and make major investments in new technology, according to documents filed on Sunday.

US import duties of 46% on goods from Vietnam, where most of iRobot's devices for the American market are made, increased its costs by $23m (£17.2m) this year, the firm said.

The loss-making company was valued at $3.56bn in 2021 after the pandemic helped to drive strong demand for its products. It is now valued at around $140m.

On Friday, iRobot's shares fell by more than 13% on the technology-heavy Nasdaq trading platform in New York.

iRobot said the bankruptcy filing was not expected to disrupt its app, supply chains or product support.

Founded in 1990 by three members of the Massachusetts Institute of Technology's (MIT) Artificial Intelligence Lab, iRobot initially focused on defence and space technology before launching the Roomba in 2002.

The Roomba holds about 42% of the US market share and 65% of the Japanese market share for robotic vacuum cleaners, according to the company.

Last year, a planned $1.7bn takeover deal by online retail giant Amazon was derailed by the European Union's competition watchdog.

Trade tariffs imposed by US Donald Trump on goods entering America from overseas has added to costs to many businesses, including iRobot, which rely on imports for product manufacturing.

Trump has argued that the import taxes will boost American jobs and industry.

Picea is a manufacturer of robotic vacuum cleaners, with research and development and production facilities in China and Vietnam.

It has more than 7,000 employees worldwide and has sold more than 20 million robotic vacuum cleaners."""

## Defining the LLM output

Now, let's define the Pydantic schema for the LLM to extract from each article.

> **Exercise** 📝
>
> - Fill in the TODO's below in the `ArticleInfo` Pydantic model.
> - Subsequently, fill in the prompt in the next section to extract the information from an article.
>

In [5]:
from typing import List, Literal
from pydantic import BaseModel, Field

In [ ]:
class BusinessSpecificInfo(BaseModel):
    business: str = Field(, description="The business or company involved")
    stock_price_change: Literal["increase", "decrease", "none"] = Field(
        ..., description="""
        Possible stock price change as result of the article. 
        - "increase" if article speaks positively about the business
        - "decrease" if article speaks negatively about the business
        - "none" if article speaks neutrally about the business
        """
    )
    reason: str = Field(..., description="A single sentence reason for the possible stock price change")
    relevant_substring: str = Field(
        ..., description="A relevant substring from the article supporting the reason (10-20 words)"
    )

class ArticleInfo(BaseModel):
    title: str = Field(..., description="The title of the article")
    summary: str = Field(..., description="A single sentence summary of the article")
    is_about_business: ...  # TODO: Fill me in! This should be a boolean (use Field here)
    business_info: ...  # TODO: Fill me in! This should be a List of BusinessSpecificInfo objects (no need to use Field here)

PydanticSchemaGenerationError: Unable to generate pydantic-core schema for Ellipsis. Set `arbitrary_types_allowed=True` in the model_config to ignore this error or implement `__get_pydantic_core_schema__` on your type to fully support it.

If you got this error by calling handler(<some type>) within `__get_pydantic_core_schema__` then you likely need to call `handler.generate_schema(<some type>)` since we do not call `__get_pydantic_core_schema__` on `<some type>` otherwise to avoid infinite recursion.

For further information visit https://errors.pydantic.dev/2.7/u/schema-for-unknown-type

In [ ]:
# %load ../solutions/use-case-introduction/pydantic-model.py

## Extracting the information

In [ ]:
from llmops_training.news_reader.generation import generate_object

In [3]:
prompt_template = "Extract structured information from the following article:\n\n{article}"

prompt = prompt_template.format(article=article)


NameError: name 'article' is not defined

In [ ]:
# %load ../solutions/use-case-introduction/make-prompt.py

Let's see if that worked!

In [ ]:
output = generate_object(prompt, ArticleInfo)

In [ ]:
output.model_dump()

Extra: check `generate_object` and figure out how to try different models! Try some different ones and see which one works well for this task.

> Tip: Use `model_name` and try for example `gpt-5`.

## (Bonus) Deep dive Instructor

> **Exercise** 📝
>
> - Step into the underlying code from `generate_object` and try to understand how Instructor extracts structured information from the text given a Pydantic base model.
> - Does it use [Function Calling](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/function-calling)? Does it do retry iterations? Could we implement this ourselves? Discuss with your peers!

---